In [1]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [3]:
print("msno")
msno_values = data.msno.unique()
print("end msno")
del(data)

msno
end msno


In [85]:
user_logs = pd.read_csv('raw_data/user_logs.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
user_logs_2 = pd.read_csv('raw_data/user_logs_v2.csv', usecols=[0], dtype={"msno":str}, engine='c', header=0)
#user_logs = pd.concat((user_logs, user_logs_2))

KeyboardInterrupt: 

In [ ]:
user_logs.head()

In [ ]:
counts = user_logs["msno"].value_counts()
counts.to_csv("cleaned_input/counts.csv")

In [141]:
del(user_logs)
del(user_logs_2)

NameError: name 'user_logs' is not defined

In [4]:
counts = pd.read_csv("cleaned_input/counts.csv")

In [5]:
#get number of unique visits
user_features = pd.DataFrame() 

user_features['msno'] = np.nan #counts.index
#user_features["frequency"] = np.nan #counts.values
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29


In [6]:
#read in user_logs
import time
chunksize = 25000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    start_time = time.time()
    chunk = chunk[chunk['msno'].isin(msno_values)]
    rows = chunk.msno.unique()
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    #print("processing ",i,"took %s seconds" % (time.time() - start_time))
    #print(len(rows))
    for row in rows:
        row_time = time.time()
        df = chunk[chunk["msno"] == row]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        differences = df.date.diff()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gap_days = differences.sum()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        average_gap_days = differences.mean()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        if len(differences.nlargest(1)) > 0:
            longest_gap = differences.nlargest(1).iloc[0].total_seconds()
        else:
            longest_gap = -1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
    #del(chunk)
    i += 1 
    print("Round ",i,"took %s seconds" % (time.time() - start_time))
    if i % 100 == 0:
        user_features.to_csv(str(i) + "_user_logs_features.csv")
user_features.head()

Round  1 took 9.102919101715088 seconds
Round  2 took 8.87452483177185 seconds
Round  3 took 7.203169107437134 seconds
Round  4 took 10.1404390335083 seconds
Round  5 took 9.145310878753662 seconds
Round  6 took 7.129753112792969 seconds
Round  7 took 10.263416051864624 seconds
Round  8 took 9.535670757293701 seconds
Round  9 took 8.722302198410034 seconds
Round  10 took 11.305909872055054 seconds
Round  11 took 9.381104230880737 seconds
Round  12 took 11.706729888916016 seconds
Round  13 took 12.570821762084961 seconds
Round  14 took 9.213936805725098 seconds
Round  15 took 13.310916185379028 seconds
Round  16 took 12.744801044464111 seconds
Round  17 took 9.922800064086914 seconds
Round  18 took 13.268102169036865 seconds
Round  19 took 12.01680612564087 seconds
Round  20 took 12.189780950546265 seconds
Round  21 took 16.857528686523438 seconds
Round  22 took 12.713438987731934 seconds
Round  23 took 14.770125150680542 seconds
Round  24 took 16.44967794418335 seconds
Round  25 took 1

KeyboardInterrupt: 

In [161]:
user_features.to_csv("cleaned_input/user_logs_features_2.csv")

In [ ]:
chunksize = 50000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs_2.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    start_time = time.time()
    chunk = chunk[chunk['msno'].isin(msno_values)]
    rows = chunk.msno.unique()
    chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
    #print("processing ",i,"took %s seconds" % (time.time() - start_time))
    #print(len(rows))
    for row in rows:
        row_time = time.time()
        df = chunk[chunk["msno"] == row]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        differences = df.date.diff()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gap_days = differences.sum()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        average_gap_days = differences.mean()
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        if len(differences.nlargest(1)) > 0:
            longest_gap = differences.nlargest(1).iloc[0].total_seconds()
        else:
            longest_gap = -1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        gaps_longer_29 = 0
        for val in differences:
            if val > pd.Timedelta('29 days'):
                gaps_longer_29 += 1
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
        user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
        #print("row ",i,"took %s seconds" % (time.time() - row_time))
    #del(chunk)
    i += 1 
    print("Round ",i,"took %s seconds" % (time.time() - start_time))
user_features.head()

In [ ]:
#combine duplicates? 

In [ ]:
user_features.head()

In [ ]:
user_features.to_csv("user_logs_features.csv")

In [ ]:
#combine what you 